This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

In [2]:
import pandas as pd 
import numpy as np
import datetime

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [3]:
file_name = "./data/data_000637.txt"
N=12000
data = pd.read_csv(file_name,nrows=N)
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
11995,1,0,139,3869200286,3251,0
11996,1,0,62,3869200286,3246,4
11997,1,0,58,3869200286,3246,11
11998,1,0,61,3869200286,3251,17


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [4]:
bx = data['BX_COUNTER']
maximun = max(bx)
print("the estimate of value x correspond to :",maximun)
#that correspond to the maximun value that we found in the column correspond to BX_COUNTER

the estimate of value x correspond to : 3563


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [5]:
orb = data['ORBIT_CNT']
total_orbit = orb[len(orb)-1]-orb[0]
total_time = (maximun+1) * 25 * total_orbit
print(datetime.timedelta(total_time*10**(-9)))


0:15:16.090560


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [6]:
data['ABS_TIME'] = round(data['TDC_MEAS'] * 25 / 30 + data['BX_COUNTER'] * 25 + maximun*25*(data['ORBIT_CNT']-3869200167),2)     
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,ABS_TIME
0,1,0,123,3869200167,2374,26,59371.67
1,1,0,124,3869200167,2374,27,59372.50
2,1,0,63,3869200167,2553,28,63848.33
3,1,0,64,3869200167,2558,19,63965.83
4,1,0,64,3869200167,2760,25,69020.83
...,...,...,...,...,...,...,...
11995,1,0,139,3869200286,3251,0,10681200.00
11996,1,0,62,3869200286,3246,4,10681078.33
11997,1,0,58,3869200286,3246,11,10681084.17
11998,1,0,61,3869200286,3251,17,10681214.17


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [7]:
tdc_channels = data.groupby('TDC_CHANNEL').size()
n_cha = tdc_channels.sort_values(ascending=False)
n_cha[0:3]


TDC_CHANNEL
139    1025
64      609
63      600
dtype: int64

6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [8]:
orbit_channels = data.groupby('ORBIT_CNT').max()
count = 0
for i in orbit_channels['TDC_CHANNEL']:
    if i == 139:
        count+=1
print("the number of unique orbits with at least one measurement from TDC_CHANNEL=139 are: ",count)


the number of unique orbits with at least one measurement from TDC_CHANNEL=139 are:  120


7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel